In [ ]:
# 라이브러리 로드

# 기본
import pandas as pd
import numpy as np

# API 용
import requests, json
import config

# 시각화
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import koreanize_matplotlib

# 지도 시각화
import folium
from folium.plugins import MarkerCluster

In [ ]:
# 그래프에 retina display 적용
%config InlineBackend.figure_format = 'retina'

# 기본 데이터 전처리

## 기본 데이터 불러오기

In [ ]:
file_path = "./data/"
df_kp20 = pd.read_csv(f"{file_path}KP2020.csv", encoding="cp949")
df_kp21 = pd.read_csv(f"{file_path}KP2021.csv", encoding="cp949")
df_npa20 = pd.read_csv(f"{file_path}NPA2020.csv", encoding="cp949")

### KP 데이터 확인, 병합 및 전처리

변수 정리
   
RECV_DEPT_NM : 접수 부서 이름   
RECV_CPLT_DM : 접수 완료 날짜, 시간   
NPA_CL : 경찰철 코드   
EVT_STAT_CD	: 사건 상태 코드   
EVT_CL_CD : 사건 종별 코드   
RPTER_SEX : 신고자 성별   
HPPN_PNU_ADDR : 발생 주소   
HPPN_X : 위도   
HPPN_Y : 경도   
SME_EVT_YN : 동일 사건 여부

In [ ]:
df_kp20.shape, df_kp21.shape

In [ ]:
display(df_kp20.head(), df_kp21.head())

In [ ]:
# kp 데이터셋 병합
df_kp = pd.concat([df_kp20, df_kp21], ignore_index=True)

In [ ]:
# 병합 잘 되었는지 확인
print(df_kp.shape, df_kp20.shape[0] + df_kp21.shape[0])
df_kp

In [ ]:
# 날짜 시간 데이터 전처리
df_kp['RECV_CPLT_DM'] = df_kp['RECV_CPLT_DM'].str.split('.').str[0]
df_kp['RECV_CPLT_DM'] = pd.to_datetime(df_kp['RECV_CPLT_DM'], format="%y/%m/%d %H:%M:%S")

# 확인하기
df_kp["RECV_CPLT_DM"].head(1)

In [ ]:
# 컬럼명 변경
df_kp = df_kp.rename(columns={'HPPN_PNU_ADDR':'HPPN_ADDR'})

### NPA 데이터 확인 및 전처리

변수 정리
   
RECV_CPLT_DT : 접수 완료 날짜   
RECV_CPLT_TM : 접수 완료 시간   
NPA_CL : 경찰청 코드   
EVT_STAT_CD : 사건 상태 코드   
EVT_CL_CD : 사건 종별 코드   
RPTER_SEX : 신고자 성별   
HPPN_OLD_ADDR : 발생 주소   
HPPN_X : 위도   
HPPN_Y : 경도   
SME_EVT_YN : 동일 사건 여부   

In [ ]:
df_npa20.shape

In [ ]:
df_npa20.head()

In [ ]:
# 시간데이터 전처리
df_npa20["RECV_CPLT_TM"] = df_npa20["RECV_CPLT_TM"].astype('str')
df_npa20["RECV_CPLT_TM"] = df_npa20["RECV_CPLT_TM"].str.rjust(width=6, fillchar='0')

# 확인하기
df_npa20.head(1)

In [ ]:
# RECV_CPLT_DM (날짜 + 시간) 파생변수 생성
df_npa20['RECV_CPLT_DM'] = df_npa20['RECV_CPLT_DT'].astype('str') + df_npa20['RECV_CPLT_TM'].astype('str')
df_npa20['RECV_CPLT_DM'] = pd.to_datetime(df_npa20['RECV_CPLT_DM'])

# 확인하기
df_npa20['RECV_CPLT_DM'].head(1)

In [ ]:
# 필요없는 데이터 삭제
df_npa20 = df_npa20.drop(labels=['RECV_CPLT_DT', 'RECV_CPLT_TM'], axis=1)

In [ ]:
# 경찰청 코드 변환하기
df_npa20['NPA_CL'].value_counts()
df_npa20['RECV_DEPT_NM'] = ['대전청' if x == 13 else '충남청' for x in df_npa20['NPA_CL']]

In [ ]:
# KP데이터에 맞춰서 컬럼명 변경
df_npa20 = df_npa20.rename(columns={'HPPN_OLD_ADDR':'HPPN_ADDR'})

## KP, NPA 데이터 병합

In [ ]:
# shape 값 확인하기
df_npa20.shape, df_kp.shape

In [ ]:
# 병합하기
df = pd.concat([df_kp, df_npa20], ignore_index=True)

# 확인하기
print(df.shape)
df.head()

In [ ]:
# 중복 삭제
df = df.drop_duplicates()

# 확인하기
df.shape

## 코드 데이터 변환

In [ ]:
# 코드북 엑셀파일 불러오기
codeBook = pd.read_excel('./data/codeBook_v3.xlsx')

# 확인하기
codeBook.head()

### 경찰청 (NPA_CL)

In [ ]:
# 딕셔너리로 만들기
npa_cl_code_dict = dict(zip(codeBook.loc[codeBook['컬럼명'] == 'NPA_CL']['코드값'].astype(str).tolist(), codeBook.loc[codeBook['컬럼명'] == 'NPA_CL']['코드명'].tolist()))

# 확인하기
npa_cl_code_dict

In [ ]:
# 딕셔너리 적용하기
df['NPA_CL'] = df['NPA_CL'].astype('str').map(npa_cl_code_dict)

# 확인하기
df['NPA_CL'].value_counts()

### 사건상태코드 (EVT_STAT_CD)

In [ ]:
# 딕셔너리로 만들기
evt_stat_code_dict = dict(zip(codeBook.loc[codeBook['컬럼명'] == 'EVT_STAT_CD']['코드값'].astype(str).tolist(), codeBook.loc[codeBook['컬럼명'] == 'EVT_STAT_CD']['코드명'].tolist()))

# 확인하기
evt_stat_code_dict

In [ ]:
# 딕셔너리 적용하기
df['EVT_STAT_CD'] = df['EVT_STAT_CD'].astype('str').map(evt_stat_code_dict)

### 사건종별코드 (EVT_CL_CD)

In [ ]:
# 딕셔너리 생성하기
evt_cl_code_dict = dict(zip(codeBook.loc[codeBook['컬럼명'] == 'EVT_CL_CD']['코드값'].astype(str).tolist(), codeBook.loc[codeBook['컬럼명'] == 'EVT_CL_CD']['코드명'].tolist()))

# 확인하기
evt_cl_code_dict

In [ ]:
# 적용하기
df['EVT_CL_CD'] = df['EVT_CL_CD'].astype('str').map(evt_cl_code_dict)

### 성별 (RPTER_SEX)

In [ ]:
def sex_sub(n):
    if n == 1:
        return '남성'
    elif n == 2:
        return '여성'
    else:
        return '불상'

In [ ]:
df['RPTER_SEX'] = df['RPTER_SEX'].map(sex_sub)

# 확인하기
df['RPTER_SEX'].value_counts()

## 컬럼명 변경

In [ ]:
df.columns = ['접수부서', '접수완료일시', '경찰청구분', '사건상태', '사건종류', '신고자 성별', '발생지점', '발생지 위도', '발생지 경도', '동일사건여부']

# 확인하기
df.head()

## 파생변수 생성

In [ ]:
df["연도"] = df["접수완료일시"].dt.year
df["월"] = df["접수완료일시"].dt.month
df['연월'] = df['접수완료일시'].map(lambda x : str(x)[:7])
df["요일"] = df["접수완료일시"].dt.day_name()
df["시간대"] = df["접수완료일시"].dt.hour
df['일자'] = df['접수완료일시'].dt.date

In [ ]:
# 정렬하기
df = df.sort_values('접수완료일시')

In [ ]:
# 요일 한국어로 바꾸기
dayofweek_dict = {'Monday' : '월',
                  'Tuesday' : '화',
                  'Wednesday' : '수',
                  'Thursday' : '목',
                  'Friday' : '금',
                  'Saturday' : '토',
                  'Sunday' : '일'
                  }

df['요일'] = df['요일'].map(dayofweek_dict)

In [ ]:
# 확인하기
df.head()

## 접수부서 합치기

In [ ]:
def dept_replace(x):
    if '대전' in x:
        return '대전청'
    elif '세종' in x:
        return '세종청'
    else:
        return '충남청'

In [ ]:
df['접수부서'] = df['접수부서'].map(dept_replace)

# 확인하기
df['접수부서'].value_counts()

## 보이스피싱 데이터 가져오기

In [ ]:
# 사건 종류가 보이스피싱인 데이터만 가져오기
df = df[df['사건종류'] == '보이스피싱']

In [ ]:
print(df.shape)
df.head()

In [ ]:
# 발생지 위도 없는 것들 없애기
df = df.dropna(subset='발생지 위도')

# 발생지점 없는 부분 삭제하기
df_get_address = df[df['발생지점'].isnull()]

# 확인하기
df_get_address.shape

## Kakao API 사용하여 Reverse Geocoding 하기

In [ ]:
# API 불러오는 함수
def get_address(lon, lat):
    url = f'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x={lon}&y={lat}'
    headers = {"Authorization": config.kakao_api_key}
    api_json = requests.get(url, headers=headers)
    full_address = json.loads(api_json.text)

    return full_address

# 역지오코딩 함수
def reverse_geocode(lon, lat):
    # json request
    try:
        json_req = get_address(lon, lat)
        json_doc = json_req.get('documents')[0]
        json_address = json_doc.get('address_name')
    except:
        json_address = np.nan

    return json_address

In [ ]:
# 함수 확인하기
get_address(df_get_address['발생지 위도'][4743], df_get_address['발생지 경도'][4743])

In [ ]:
# 적용하기
df_get_address.loc['발생지점'] = df_get_address.apply(lambda row : reverse_geocode(row['발생지 위도'], row['발생지 경도']), axis=1)

# 확인하기
df_get_address.head()

In [ ]:
# 여전히 안 불러와진 부분 삭제
df = df.dropna(subset='발생지점')

In [ ]:
# 다시 합치기
df = pd.concat([df, df_get_address])

## 대전광역시, 충청남도, 세종특별자치시 데이터

In [ ]:
df = df[df['발생지점'].str.contains("대전광역시|충청남도|세종특별자치시")]
df.shape

In [1]:
# 발생지점 행정구역 파생변수 생성
sigungu_list = []
for idx in list(df.index):

    # 세종특별자치시의 경우 읍면동을 행정구역으로 정의한다.
    if df['발생지점'][idx].split(' ')[0] == '세종특별자치시':
        a = df['발생지점'][idx].split(' ')[2]
    
    # 대전광역시, 충청남도의 경우 시군구를 행정구역으로 정의한다.
    else:
        a = df['발생지점'][idx].split(' ')[1]
    
    sigungu_list.append(a)

# 파생변수에 입력
df['발생시군구'] = sigungu_list

# 오류 없는지 확인
df['발생시군구'].isnull().sum()

In [ ]:
# 시도 가져오기
sido_list = []
for idx in list(df.index):
    a = df['발생지점'][idx].split(' ')[0]
    
    sido_list.append(a)

# 파생변수에 입력
df['발생시도'] = sido_list

# 오류 없는지 확인
df['발생시군구'].isnull().sum()

# 보이스피싱 피해, 인구 전처리

## 인구 데이터

In [ ]:
file_path_population = f'{file_path}20_22_population/'
df_ppl_cn = pd.read_csv(f'{file_path_population}chungnam_population_20_22.csv', encoding='cp949')
df_ppl_dj = pd.read_csv(f'{file_path_population}daejeon_population_20_22.csv', encoding='cp949')
df_ppl_sj = pd.read_csv(f'{file_path_population}sejong_population_20_22.csv', encoding='cp949')

In [ ]:
# 확인하기
display(df_ppl_cn.head(), df_ppl_dj.head(), df_ppl_sj.head())

### 충청남도 인구 데이터 전처리

In [ ]:
# 가져올 컬럼 리스트 생성
col = [0]
for n in range(1, 40, 13):
    col.append(n)

# 가져올 충청남도 데이터 row 리스트 생성
row_cn = [1]
for n in range(4, 18):
    row_cn.append(n)

In [ ]:
df_ppl_cn = df_ppl_cn.iloc[row_cn, col]
df_ppl_cn['행정구역'] = df_ppl_cn['행정구역'].str.split(' ').str[1]
df_ppl_cn.columns = ['행정구역', '2020 인구수', '2021 인구수', '2022 인구수']

# 확인하기
df_ppl_cn

### 대전광역시 인구 데이터 전처리

In [ ]:
# 가져올 대전광역시 데이터 row 리스트 생성
row_dj = []
for n in range(1, 6):
    row_dj.append(n)

In [ ]:
df_ppl_dj = df_ppl_dj.iloc[row_dj, col]
df_ppl_dj['행정구역'] = df_ppl_dj['행정구역'].str.split(' ').str[1]
df_ppl_dj.columns = ['행정구역', '2020 인구수', '2021 인구수', '2022 인구수']

# 확인하기
df_ppl_dj

### 세종특별자치시 인구 데이터 전처리

In [ ]:
# 가져올 세종특별자치시 데이터 row 리스트 생성
row_sj = []
for n in range(1, 23):
    row_sj.append(n)

In [ ]:
df_ppl_sj = df_ppl_sj.iloc[row_sj, col]
df_ppl_sj['행정구역'] = df_ppl_sj['행정구역'].str.split(' ').str[2].str.split('(').str[0]
df_ppl_sj.columns = ['행정구역', '2020 인구수', '2021 인구수', '2022 인구수']
df_ppl_sj = df_ppl_sj.drop(index=[14, 19])

# 확인하기
df_ppl_sj

## 보이스피싱 데이터

In [ ]:
# 보이스피싱 발생시도별로 나누기
df_vp_cn = df[df['발생시도'] == '충청남도']
df_vp_dj = df[df['발생시도'] == '대전광역시']
df_vp_sj = df[df['발생시도'] == '세종시']

In [ ]:
# 확인하기
display(df_vp_cn.head(), df_vp_dj.head(), df_vp_sj.head())

### 충청남도 보이스피싱 데이터 전처리

In [ ]:
df_vp_cn = df_vp_cn.groupby(['발생시군구', '연도'])[['경찰청구분']].count()
df_vp_cn = df_vp_cn.unstack()
df_vp_cn = df_vp_cn.reset_index()
df_vp_cn = df_vp_cn.iloc[:, :4]

# 컬럼명 변경
df_vp_cn.columns = ['행정구역', '2020 보이스피싱 신고 건수', '2021 보이스피싱 신고 건수', '2022 보이스피싱 신고 건수']

In [ ]:
# 확인하기
df_vp_cn

### 대전광역시 보이스피싱 데이터 전처리

In [ ]:
df_vp_dj = df_vp_dj.groupby(['발생시군구', '연도'])[['경찰청구분']].count()
df_vp_dj = df_vp_dj.unstack()
df_vp_dj = df_vp_dj.reset_index()
df_vp_dj = df_vp_dj.iloc[:, :4]

# 컬럼명 변경
df_vp_dj.columns = ['행정구역', '2020 보이스피싱 신고 건수', '2021 보이스피싱 신고 건수', '2022 보이스피싱 신고 건수']

In [ ]:
# 확인하기
df_vp_dj

### 세종특별자치시 보이스피싱 데이터 전처리

In [ ]:
# 조치원읍을 제외한 다른 주소들에 괄호 등 불필요한 요소 제거
df_vp_sj.loc[df_vp_sj['발생시군구'] != '조치원읍', '발생시군구'] = df_vp_sj.loc[df_vp_sj['발생시군구'] != '조치원읍', '발생시군구'].apply(lambda x : x[:3])

df_vp_sj = df_vp_sj.groupby(['발생시군구', '연도'])[['경찰청구분']].count()
df_vp_sj = df_vp_sj.unstack()
df_vp_sj = df_vp_sj.reset_index()
df_vp_sj = df_vp_sj.iloc[:, :4]

# 결측치가 존재하므로 제거
df_vp_sj = df_vp_sj.dropna()

# 컬럼명 변경
df_vp_sj.columns = ['행정구역', '2020 보이스피싱 신고 건수', '2021 보이스피싱 신고 건수', '2022 보이스피싱 신고 건수']

In [ ]:
# 확인하기
df_vp_sj

## 데이터셋 합치기
* 정석 방식으로 연도라는 컬럼을 만들어서 코드로 전처리를 진행하려 하였다.  
* 시간관계상 여기까지 전처리한 부분을 merge하여 저장하고 엑셀이나 구글시트를 사용하여 위 방식을 빠르게 진행하였다. 
* 연도 컬럼을 만들고 인구수 데이터는 인구수 데이터끼리, 보이스피싱 신고 건수 데이터는 보이스피싱 신고 건수 끼리 각각 하나의 컬럼으로 합쳐주었다.

### 충청남도 데이터셋

In [ ]:
df_cn = pd.merge(df_vp_cn, df_ppl_cn, on='행정구역', how='inner')

# 확인하기
df_cn

In [ ]:
# 저장
df_cn.to_csv(f'{file_path}충청남도_20_22_초안.csv')

In [ ]:
# 엑셀이나 구글 시트를 사용하여 전처리 마무리한 파일 불러오기
df_cn = pd.read_csv(f'{file_path}충청남도_20_22.csv')

# 확인하기
df_cn

### 대전광역시 데이터셋

In [ ]:
df_dj = pd.merge(df_vp_dj, df_ppl_dj, on='행정구역', how='inner')

# 확인하기
df_dj

In [ ]:
# 저장
df_dj.to_csv(f'{file_path}대전광역시_20_22_초안.csv')

In [ ]:
# 엑셀이나 구글 시트를 사용하여 전처리 마무리한 파일 불러오기
df_dj = pd.read_csv(f'{file_path}대전광역시_20_22.csv')

# 확인하기
df_dj

### 세종특별자치시 데이터셋

In [ ]:
df_sj = pd.merge(df_vp_sj, df_ppl_sj, on='행정구역', how='inner')

# 확인하기
df_sj

In [ ]:
# 저장
df_sj.to_csv(f'{file_path}세종특별자치시_20_22_초안.csv')

In [ ]:
# 엑셀이나 구글 시트를 사용하여 전처리 마무리한 파일 불러오기
df_sj = pd.read_csv(f'{file_path}세종특별자치시_20_22.csv')

# 확인하기
df_sj

## 인구수 대비 보이스피싱 신고 비율 구하기

In [ ]:
# 데이터셋 확인하기
display(df_cn.head(), df_dj.head(), df_sj.head())

In [ ]:
# 인구수 데이터에서 ',' 삭제하기
df_cn['인구수'] = df_cn['인구수'].str.replace(',', '')
df_dj['인구수'] = df_dj['인구수'].str.replace(',', '')
df_sj['인구수'] = df_sj['인구수'].str.replace(',', '')

# 모든 수치형 데이터 integer로 타입 변경
df_cn['인구수'] = df_cn['인구수'].astype('int')
df_dj['인구수'] = df_dj['인구수'].astype('int')
df_sj['인구수'] = df_sj['인구수'].astype('int')

df_cn['보이스피싱 신고 건수'] = df_cn['보이스피싱 신고 건수'].astype('int')
df_dj['보이스피싱 신고 건수'] = df_dj['보이스피싱 신고 건수'].astype('int')
df_sj['보이스피싱 신고 건수'] = df_sj['보이스피싱 신고 건수'].astype('int')

In [ ]:
# 비율 계산하기
df_cn['보이스피싱 신고 비율'] = df_cn['보이스피싱 신고 건수']/df_cn['인구수']*100
df_dj['보이스피싱 신고 비율'] = df_dj['보이스피싱 신고 건수']/df_dj['인구수']*100
df_sj['보이스피싱 신고 비율'] = df_sj['보이스피싱 신고 건수']/df_sj['인구수']*100

In [ ]:
# 컬럼 순서 정렬
df_cn = df_cn[['행정구역', '보이스피싱 신고 건수', '인구수', '보이스피싱 신고 비율', '연도']]
df_dj = df_dj[['행정구역', '보이스피싱 신고 건수', '인구수', '보이스피싱 신고 비율', '연도']]
df_sj = df_sj[['행정구역', '보이스피싱 신고 건수', '인구수', '보이스피싱 신고 비율', '연도']]

In [ ]:
# 확인하기
display(df_cn.head(), df_dj.head(), df_sj.head())

# 지역별 EDA

In [ ]:
# 팔레트 색 지정
colors = ['#9AC4DA', '#6B9EC6', '#3A566B']
sns.set_palette(sns.color_palette(colors))

## 충청남도

In [ ]:
plt.figure(figsize=(10, 15))

plt.subplot(3, 1, 1)
sns.barplot(data=df_cn, x='행정구역', y='보이스피싱 신고 비율', hue='연도')
plt.title('충청남도 행정구열별 인구수 대비 보이스피싱 신고 비율', fontsize = 10)

plt.subplot(3, 1, 2)
sns.barplot(data=df_cn, x='행정구역', y='보이스피싱 신고 건수', hue='연도')
plt.title('충청남도 행정구열별 보이스피싱 신고 건수', fontsize = 10)

plt.subplot(3, 1, 3)
sns.barplot(data=df_cn, x='행정구역', y='인구수', hue='연도')
plt.title('충청남도 행정구열별 인구수', fontsize = 10);

## 대전광역시

In [ ]:
plt.figure(figsize=(18, 5))

plt.subplot(1, 3, 1)
sns.barplot(data=df_dj, x='행정구역', y='보이스피싱 신고 비율', hue='연도')
plt.title('대전광역시 행정구열별 인구수 대비 보이스피싱 신고 비율', fontsize = 10)

plt.subplot(1, 3, 2)
sns.barplot(data=df_dj, x='행정구역', y='보이스피싱 신고 건수', hue='연도')
plt.title('대전광역시 행정구열별 보이스피싱 신고 건수', fontsize = 10)

plt.subplot(1, 3, 3)
sns.barplot(data=df_dj, x='행정구역', y='인구수', hue='연도')
plt.title('대전광역시 행정구열별 인구수', fontsize = 10);

## 세종특별자치시

In [ ]:
plt.figure(figsize=(15, 15))

plt.subplot(3, 1, 1)
sns.barplot(data=df_sj, x='행정구역', y='보이스피싱 신고 비율', hue='연도')
plt.title('세종특별자치시 행정구열별 인구수 대비 보이스피싱 신고 비율', fontsize = 10)

plt.subplot(3, 1, 2)
sns.barplot(data=df_sj, x='행정구역', y='보이스피싱 신고 건수', hue='연도')
plt.title('세종특별자치시 행정구열별 보이스피싱 신고 건수', fontsize = 10)

plt.subplot(3, 1, 3)
sns.barplot(data=df_sj, x='행정구역', y='인구수', hue='연도')
plt.title('세종특별자치시 행정구열별 인구수', fontsize = 10);

# 지도 시각화

## 데이터 전처리

### 충청남도

In [ ]:
df_cn_20 = df_cn[df_cn['연도'] == '2020'][['행정구역', '보이스피싱 신고 비율']]
df_cn_21 = df_cn[df_cn['연도'] == '2021'][['행정구역', '보이스피싱 신고 비율']]
df_cn_22 = df_cn[df_cn['연도'] == '2022'][['행정구역', '보이스피싱 신고 비율']]

In [ ]:
# 확인하기
display(df_cn_20.head(), df_cn_21.head(), df_cn_22.head())

### 대전광역시

In [ ]:
df_dj_20 = df_dj[df_dj['연도'] == '2020'][['행정구역', '보이스피싱 신고 비율']]
df_dj_21 = df_dj[df_dj['연도'] == '2021'][['행정구역', '보이스피싱 신고 비율']]
df_dj_22 = df_dj[df_dj['연도'] == '2022'][['행정구역', '보이스피싱 신고 비율']]

In [ ]:
# 확인하기
display(df_dj_20.head(), df_dj_21.head(), df_dj_22.head())

### 세종특별자치시

In [ ]:
df_sj_20 = df_sj[df_sj['연도'] == '2020'][['행정구역', '보이스피싱 신고 비율']]
df_sj_21 = df_sj[df_sj['연도'] == '2021'][['행정구역', '보이스피싱 신고 비율']]
df_sj_22 = df_sj[df_sj['연도'] == '2022'][['행정구역', '보이스피싱 신고 비율']]

In [ ]:
# 확인하기
display(df_sj_20.head(), df_sj_21.head(), df_sj_22.head())

In [ ]:
# folium 사용시 세종특별자치시는 이름 전에 '세종특별자치시 세종시'가 있어야 인식됨
for x in [df_sj_20, df_sj_21, df_sj_22]:
    for idx in list(x.index):
        a = x.loc[idx, '지역별']
        x.loc[idx, '지역별'] = f'세종특별자치시 세종시 {a}'

## choropleth 시각화

In [ ]:
# choropleth에 필요한 geojson 파일 가져오기
# 충남, 대전은 시군구 파일 사용
cn_dj_geo = json.load(open(f'{file_path}geojson/TL_SCCO_SIG.json', encoding='utf-8'))
# 세종은 세종특별자치시 읍면동 파일 사용
sj_geo = json.load(open(f'{file_path}geojson/hangjeongdong_세종특별자치시.geojson', encoding='utf-8'))

In [2]:
html_path = './folium_html/'

### 충청남도

In [ ]:
cn_20_m = folium.Map(
          location=[36.5184, 126.8000],
          zoom_start=9
)

folium.GeoJson(
    cn_dj_geo,
    name='SIG_KOR_NM'
).add_to(cn_20_m)

cn_20_m.choropleth(geo_data=cn_dj_geo,
                   data=df_cn_20, 
                   columns = ['지역별', '보이스피싱 신고 비율'],
                   fill_color='YlOrRd',
                   fill_opacity=0.5,
                   line_opacity=0.2,
                   key_on='properties.SIG_KOR_NM',
                   legend_name="충청남도 2020 보이스피싱 신고 비율"
                   )

cn_20_m.save(f'{html_path}cn_20.html')

### 대전광역시

In [ ]:
dj_20_m = folium.Map(
          location=[36.3504, 127.3845],
          zoom_start=11
)

folium.GeoJson(
    cn_dj_geo,
    name='SIG_KOR_NM'
).add_to(dj_20_m)

dj_20_m.choropleth(geo_data=cn_dj_geo,
                   data=df_dj_20, 
                   columns = ['지역별', '보이스피싱 신고 비율'],
                   fill_color='YlOrRd',
                   fill_opacity=0.5,
                   line_opacity=0.2,
                   key_on='properties.SIG_KOR_NM',
                   legend_name="대전광역시 2020 보이스피싱 신고 비율"
                   )

dj_20_m.save('./folium_html/dj_20.html')